In [ ]:
import os
import pandas as pd
import re
import random
import spacy
from sklearn.model_selection import train_test_split
import numpy as np

from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch import cuda, tensor
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.optim as optim
from huggingface_hub import HfFolder, Repository, create_repo

from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import BertTokenizer, BertModel
from bert_score import BERTScorer

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from transformers import Trainer, TrainingArguments

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cwd = '/content/drive/MyDrive/LLMs'
cwd

In [ ]:
filename = f'{cwd}/subreddits_train_data.parquet'
raw_df = pd.read_parquet(filename)
raw_df.head()

In [ ]:
raw_df['PostText'].values[0]

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
def shuffle_text(text):
    keywords = []
    for item in nlp(text):
        if not item.is_stop:
            keywords.append(item.text)
    keywords_para = ' '.join(keywords)

    return keywords_para

def extract_keywords(text):
    try:
        split_text = text.split("Text: ")
        text = split_text[1]
        keywords_para = shuffle_text(text)

        return {
        "text": text,
        "keywords": keywords_para
        }
    except:
        return {'text': '', 'keywords': ''}

In [ ]:
keywords = []
texts = []
for post in raw_df['PostText'].values:
    result = extract_keywords(post)
    keywords.append(result['keywords'])
    texts.append(result['text'])

In [ ]:
raw_df['PostText'].values[0]

In [ ]:
df = pd.DataFrame({'text': texts, 'keywords': keywords})
df.head()

In [ ]:
df.shape

In [ ]:
df.to_parquet(f'{cwd}/subreddits_train_data_keywords.parquet')

In [ ]:
df = pd.read_parquet(f'{cwd}/subreddits_train_data_keywords.parquet')
df.head()

In [ ]:
df['text'].values[2]

In [ ]:
df['keywords'].values[2]

In [ ]:
X = df['keywords']
y = df['text']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [ ]:
X_train.values[0]

In [ ]:
y_train.values[0]

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
device

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")
model = model.to(device)
max_source_length = 128
max_target_length = 128
task_prefix = "generate: "
input_sequences = [task_prefix + text for text in X_train.values]
encoding = tokenizer(
 input_sequences,
 padding="longest",
 max_length=max_source_length,
 truncation=True,
 return_tensors="pt",
)

In [ ]:
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
target_encoding = tokenizer(
 list(y_train.values),
 padding="longest",
 max_length=max_target_length,
 truncation=True,
 return_tensors="pt",
)

In [ ]:
labels = target_encoding.input_ids.to(device)
labels[labels == tokenizer.pad_token_id] = -100
train_dataset = TensorDataset(input_ids, attention_mask, labels)

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

accumulation_steps = 4
num_epochs = 4
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(num_epochs):
    print(f'Started with epoch: {epoch}')
    for i, (input_ids_batch, attention_mask_batch, labels_batch) in enumerate(train_loader):
        input_ids_batch = input_ids_batch.to(device)
        attention_mask_batch = attention_mask_batch.to(device)
        labels_batch = labels_batch.to(device)

        outputs = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch, labels=labels_batch)
        loss = outputs.loss

        loss = loss/accumulation_steps

        loss.backward()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

In [ ]:
loss.item()

In [ ]:
i = 6
shuffled_text = X_test.values[i]
shuffled_text

In [ ]:
y_test.values[i]

In [ ]:
shuffled_text = f"generate: {shuffled_text}"
input_ids = tokenizer(shuffled_text, return_tensors="pt").input_ids.to(device)
output = model.generate(input_ids, max_length=400)
tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
answers_fine_tuned = []
for ind in range(10):
    test_text = X_test.values[ind]
    test_text = f"generate: {test_text}"
    input_ids = tokenizer(test_text, return_tensors="pt").input_ids.to(device)
    output = model.generate(input_ids, max_length=50)
    answers_fine_tuned.append(tokenizer.decode(output[0], skip_special_tokens=True))

    print('Ground truth:', y_test.values[ind])
    print('Prediction (fine tuned model):', answers_fine_tuned[ind])
    print('Keywords:', X_test.values[ind])
    print('\n')
    print('***'*20)

In [ ]:
!huggingface-cli login

In [ ]:
token = HfFolder.get_token()
if token is None:
  raise ValueError("You must be logged into the Hugging Face CLI")

In [ ]:
repo_name = "SubredditSummarizer"
username = 'saurabhkumar3400'

repo_path = os.path.join(username, repo_name)

# create_repo(repo_path, token=HfFolder.get_token(), exist_ok=True)

In [ ]:
repo = Repository(repo_path, clone_from=f"{repo_path}", use_auth_token=True)
model.save_pretrained(repo_path)
tokenizer.save_pretrained(repo_path)
repo.push_to_hub(commit_message="sentence level, n_epochs = 4")

In [ ]:
model_repo = os.path.join(username, repo_name)
dwnld_tokenizer = T5Tokenizer.from_pretrained(model_repo)
dwnld_model = T5ForConditionalGeneration.from_pretrained(model_repo)
dwnld_model = dwnld_model.to(device)

In [ ]:
X_test.shape

In [ ]:
answers_fine_tuned = []
batch_size = 100
for test_text in X_test.values[:batch_size]:
    test_text = f"generate: {test_text}"
    input_ids = dwnld_tokenizer(test_text, return_tensors="pt").input_ids.to(device)
    output = dwnld_model.generate(input_ids, max_length=50, num_return_sequences=1)
    answers_fine_tuned.append(dwnld_tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
rouge_metric = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [ ]:
def print_rouge(rouge_n):

  fine_tuned_rouge1_prec = [rouge_metric.score(y_test.values[ind], answers_fine_tuned[ind])[rouge_n].precision for ind in range(batch_size)]

  fine_tuned_rouge1_recall = [rouge_metric.score(y_test.values[ind], answers_fine_tuned[ind])[rouge_n].recall for ind in range(batch_size)]

  fine_tuned_rouge1_fscore = [rouge_metric.score(y_test.values[ind], answers_fine_tuned[ind])[rouge_n].fmeasure for ind in range(batch_size)]

  print('Fine-tuned model (precision):', np.mean(fine_tuned_rouge1_prec))
  print('\n')
  print('Fine-tuned model (recall):', np.mean(fine_tuned_rouge1_recall))
  print('\n')
  print('Fine-tuned model (fscore):', np.mean(fine_tuned_rouge1_fscore))

In [ ]:
print_rouge('rouge1')

In [ ]:
print_rouge('rougeL')

In [ ]:
weights = (0.25, 0.25, 0, 0)  # Weights for uni-gram, bi-gram, tri-gram, and 4-gram

fine_tuned_bleu = [sentence_bleu([y_test.values[ind].split()], answers_fine_tuned[ind].split(), weights=weights, smoothing_function=SmoothingFunction().method1) for ind in range(batch_size)]
print('Fine-tuned model:', np.mean(fine_tuned_bleu))

In [ ]:
# BERTScore calculation
bert_scorer = BERTScorer(model_type='bert-base-uncased')

In [ ]:
bert_scores = [bert_scorer.score([answers_fine_tuned[ind]], [y_test.values[ind]]) for ind in range(batch_size)]
precision = [bert_scores[ind][0] for ind in range(batch_size)]
recall = [bert_scores[ind][1] for ind in range(batch_size)]
f1 = [bert_scores[ind][2] for ind in range(batch_size)]
print('Fine-tuned model (precision):', np.mean(precision))
print('Fine-tuned model (recall):', np.mean(recall))
print('Fine-tuned model (fscore):', np.mean(f1))

In [ ]:
space = {
 'learning_rate': hp.loguniform('learning_rate', -2, -1),
 'per_device_train_batch_size': hp.choice( 'per_device_train_batch_size', [8, 16, 32, 64]),
 'num_train_epochs': hp.choice('num_train_epochs', [1, 2, 3]),
 'weight_decay': hp.uniform('weight_decay', 0.0, 0.3),
}

In [ ]:
train_data = pd.DataFrame({'text': X_train.values, 'labels': y_train.values})
train_data.head()

In [ ]:
test_data = pd.DataFrame({'text': X_test.values, 'labels': y_test.values})
test_data.head()

In [ ]:
class RedditDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

    def __len__(self):
        return len(self.labels['input_ids'])

In [ ]:
train_encodings = dwnld_tokenizer(list(X_train.values), truncation=True, padding=True)
test_encodings = dwnld_tokenizer(list(X_test.values), truncation=True, padding=True)

train_label_encodings = dwnld_tokenizer(list(y_train.values), truncation=True, padding=True)
test_label_encodings = dwnld_tokenizer(list(y_test.values), truncation=True, padding=True)

In [ ]:
train_dataset = RedditDataset(train_encodings, train_label_encodings)
test_dataset = RedditDataset(test_encodings, test_label_encodings)

In [ ]:
def objective(hyperparams):
    model_repo = os.path.join(username, repo_name)
    dwnld_tokenizer = T5Tokenizer.from_pretrained(model_repo)
    dwnld_model = T5ForConditionalGeneration.from_pretrained(model_repo)
    dwnld_model = dwnld_model.to(device)

    training_args = TrainingArguments(
        output_dir=model_repo,
        learning_rate=hyperparams['learning_rate'],
        per_device_train_batch_size=hyperparams['per_device_train_batch_size'],
        num_train_epochs=hyperparams['num_train_epochs'],
        weight_decay=hyperparams['weight_decay'],
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,
        report_to=None
    )

    trainer = Trainer(
        model=dwnld_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return {'loss': eval_results["eval_loss"], 'status': STATUS_OK}

In [ ]:
pip install wandb

In [ ]:
wandb login

In [ ]:
best = fmin(
    fn=objective,
    space=space,
    max_evals=3,
    trials=Trials()
)